In [1]:
!pip install yfinance pandas requests tqdm openpyxl

In [6]:
import yfinance as yf
import pandas as pd
import requests
from tqdm import tqdm
import time
import os

# ---------- CONFIG ----------
PRICE_LIMIT = 40.0
EXCHANGES = {"NYSE", "NASDAQ", "AMEX"}   # exclude OTC/pink
OUTPUT_DIR = "./screener_output"
ALL_CSV = os.path.join(OUTPUT_DIR, "all_us_stocks_under_$40.csv")
TOP100_MCAP_CSV = os.path.join(OUTPUT_DIR, "top100_by_marketcap_under_$40.csv")
TOP100_VOL_CSV = os.path.join(OUTPUT_DIR, "top100_by_volume_under_$40.csv")
PREVIEW_ROWS = 200
BATCH_SIZE = 10
SAVE_INTERVAL = 500
# ----------------------------

os.makedirs(OUTPUT_DIR, exist_ok=True)

def get_exchange_list(exchange):
    urls = []
    if exchange == "NASDAQ":
        urls = ["https://api.nasdaq.com/api/screener/stocks?exchange=NASDAQ&download=true"]
    elif exchange == "NYSE":
        urls = ["https://api.nasdaq.com/api/screener/stocks?exchange=NYSE&download=true"]
    elif exchange == "AMEX":
        urls = ["https://api.nasdaq.com/api/screener/stocks?exchange=AMEX&download=true"]
    else:
        return []
    headers = {"User-Agent": "Mozilla/5.0"}
    for u in urls:
        try:
            r = requests.get(u, headers=headers, timeout=20)
            if r.status_code == 200:
                j = r.json()
                rows = j.get("data", {}).get("rows", [])
                tickers = [r["symbol"].strip() for r in rows if r.get("symbol")]
                if tickers:
                    return tickers
        except Exception:
            pass
    return []

def gather_all_tickers():
    tickers = set()
    for ex in EXCHANGES:
        print(f"Fetching tickers for {ex}...")
        ex_tickers = get_exchange_list(ex)
        tickers.update(ex_tickers)
        time.sleep(1)
    print(f"Total unique tickers fetched: {len(tickers)}")
    return sorted(tickers)

def is_common_stock(ticker):
    bad_patterns = ["WS", "-WS", "-W", "W", "-U", "U", "PRA", "PR", ".P", "-R", "-M", "^"]
    return not any(bp in ticker for bp in bad_patterns)

def fetch_info(ticker):
    try:
        info = yf.Ticker(ticker).info
        if not info:
            return None
        price = info.get("regularMarketPrice")
        if price is None or price >= PRICE_LIMIT:
            return None
        return {
            "Ticker": ticker,
            "Company": info.get("longName") or info.get("shortName"),
            "Exchange": info.get("exchange"),
            "LastPrice": price,
            "PercentChange": (
                (price - info.get("regularMarketPreviousClose", price)) /
                info.get("regularMarketPreviousClose", price) * 100
                if info.get("regularMarketPreviousClose") else None
            ),
            "MarketCap": info.get("marketCap"),
            "AvgDailyVolume3M": info.get("averageDailyVolume3Month") or info.get("averageVolume"),
            "Sector": info.get("sector"),
            "Industry": info.get("industry"),
            "LastTradeTime": info.get("regularMarketTime"),
            "PE": info.get("trailingPE"),
            "ForwardPE": info.get("forwardPE"),
            "52WeekHigh": info.get("fiftyTwoWeekHigh"),
            "52WeekLow": info.get("fiftyTwoWeekLow"),
            "DividendYield": info.get("dividendYield"),
        }
    except Exception:
        return None

In [7]:
def main():
    all_tickers = gather_all_tickers()
    all_tickers = [t for t in all_tickers if is_common_stock(t)]

    results = []
    counter = 0

    for i in tqdm(range(0, len(all_tickers), BATCH_SIZE)):
        batch = all_tickers[i:i+BATCH_SIZE]
        try:
            tk = yf.Tickers(" ".join(batch))
            for t in batch:
                try:
                    info = tk.tickers[t].info
                    if not info:
                        raise ValueError("empty")
                    price = info.get("regularMarketPrice")
                    if price and price < PRICE_LIMIT:
                        row = fetch_info(t)  # reuse single fetch for consistency
                        if row:
                            results.append(row)
                except Exception:
                    row = fetch_info(t)
                    if row:
                        results.append(row)
        except Exception:
            for t in batch:
                row = fetch_info(t)
                if row:
                    results.append(row)

        counter += len(batch)
        if counter % SAVE_INTERVAL == 0:
            df_temp = pd.DataFrame(results)
            df_temp.to_csv(ALL_CSV, index=False)
            print(f"Progress saved at {counter} tickers.")

        time.sleep(0.5)  # rate limit

    df = pd.DataFrame(results)

    # Ensure all columns exist
    expected_cols = ["Ticker", "Company", "Exchange", "LastPrice", "PercentChange",
                    "MarketCap", "AvgDailyVolume3M", "Sector", "Industry",
                    "LastTradeTime", "PE", "ForwardPE", "52WeekHigh", "52WeekLow", "DividendYield"]

    for col in expected_cols:
        if col not in df.columns:
            df[col] = None

    # Normalize column names: remove spaces, lowercase
    df.columns = [c.strip().replace(" ", "").lower() for c in df.columns]

    # Now sorting is safe
    df_sorted_mcap = df.sort_values(by="marketcap", ascending=False).reset_index(drop=True)
    df_sorted_vol = df.sort_values(by="avgdailyvolume3m", ascending=False).reset_index(drop=True)

    # df = pd.DataFrame(results).drop_duplicates(subset=["Ticker"])
    # print(f"Final collected rows: {len(df)}")

    # df_sorted_mcap = df.sort_values(by="MarketCap", ascending=False).reset_index(drop=True)
    # df_sorted_vol = df.sort_values(by="AvgDailyVolume3M", ascending=False).reset_index(drop=True)

    df.to_csv(ALL_CSV, index=False)
    df_sorted_mcap.head(100).to_csv(TOP100_MCAP_CSV, index=False)
    df_sorted_vol.head(100).to_csv(TOP100_VOL_CSV, index=False)

    preview = df.head(PREVIEW_ROWS)
    preview_file = os.path.join(OUTPUT_DIR, "preview_first_200_rows.csv")
    preview.to_csv(preview_file, index=False)

    print(f"\nWrote {ALL_CSV} ({len(df)} rows)")
    print(f"Wrote {TOP100_MCAP_CSV}")
    print(f"Wrote {TOP100_VOL_CSV}")
    print(f"Wrote preview file: {preview_file}")
    print("Preview of first 10 rows:")
    print(preview.head(10).to_string(index=False))

if __name__ == "__main__":
    main()


Fetching tickers for AMEX...
Fetching tickers for NYSE...
Fetching tickers for NASDAQ...
Total unique tickers fetched: 7038


  4%|▎         | 20/535 [00:56<25:28,  2.97s/it]ERROR:yfinance:HTTP Error 500: 
ERROR:yfinance:HTTP Error 500: 
ERROR:yfinance:HTTP Error 500: 
ERROR:yfinance:HTTP Error 500: 
  9%|▉         | 49/535 [02:20<23:22,  2.89s/it]

Progress saved at 500 tickers.


 12%|█▏        | 64/535 [03:03<22:14,  2.83s/it]ERROR:yfinance:HTTP Error 500: 
ERROR:yfinance:HTTP Error 500: 
ERROR:yfinance:HTTP Error 500: 
ERROR:yfinance:HTTP Error 500: 
 13%|█▎        | 69/535 [03:18<23:26,  3.02s/it]ERROR:yfinance:HTTP Error 500: 
ERROR:yfinance:HTTP Error 500: 
 15%|█▌        | 82/535 [03:56<21:33,  2.85s/it]ERROR:yfinance:HTTP Error 500: 
ERROR:yfinance:HTTP Error 500: 
ERROR:yfinance:HTTP Error 500: 
 19%|█▊        | 99/535 [04:18<08:46,  1.21s/it]

Progress saved at 1000 tickers.


...

Progress saved at 4500 tickers.


 93%|█████████▎| 499/535 [12:22<00:42,  1.19s/it]

Progress saved at 5000 tickers.


100%|██████████| 535/535 [13:04<00:00,  1.47s/it]


Wrote ./screener_output/all_us_stocks_under_$40.csv (618 rows)
Wrote ./screener_output/top100_by_marketcap_under_$40.csv
Wrote ./screener_output/top100_by_volume_under_$40.csv
Wrote preview file: ./screener_output/preview_first_200_rows.csv
Preview of first 10 rows:
ticker                         company exchange  lastprice  percentchange    marketcap  avgdailyvolume3m             sector                      industry  lasttradetime         pe  forwardpe  52weekhigh  52weeklow  dividendyield
    AA               Alcoa Corporation      NYQ    33.2400       2.942087 8606334976.0         5849754.0    Basic Materials                      Aluminum     1757707202   7.839624  10.420063     47.7700    21.5300           1.24
  AACB      Artius II Acquisition Inc.      NGM    10.1300      -0.156714  280347744.0           31004.0 Financial Services               Shell Companies     1757707200       None        NaN     10.1800     9.8500            NaN
 AACBR      Artius II Acquisition Inc.      N